In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import pandas as pd

data = pd.read_csv('train.tsv.zip', sep="\t")
data.describe()

In [8]:
import os, sys, csv, re

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

from tqdm.auto import tqdm

# import en_core_web_md

# nlp = en_core_web_md.load()


data = data.rename(columns = {'Phrase' : 'Text', 'Sentiment' : 'Score'})
data_shuffle_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for i, (train_indices, test_indices) in enumerate(data_shuffle_split.split(data, data.Score)):
    data = data.iloc[train_indices].reset_index(drop = True)

data.head()

,PhraseId,SentenceId,Text,Score
0,32387,1518,really good,4
1,61509,3110,vistas,2
2,7990,327,and not worth,2
3,65074,3292,unstinting look,2
4,102946,5424,"A thoroughly enjoyable , heartfelt",4


In [14]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn

tokenizer = RegexpTokenizer(r"['\-\w\.\?!,]+")
lemmatizer = WordNetLemmatizer()
stops = stopwords.words('english')


def tockenize_(text):
    if type(text) == list:
        text = ' '.join(text)
    text = re.sub(r" (n't|'[a-z]{1,3})", r'\1', text)
    text = re.sub(r'[^a-z0-9\s\'\-\.\?!,\"]', '', text.lower())
    text = ' '.join([lemmatizer.lemmatize(w) for w in tokenizer.tokenize(text)])
    return text


tqdm.pandas()
print('train')
data['Tokens'] = data.Text.apply(tockenize_)
data['nb_words'] = data.Tokens.str.count(' ') + 1
data['rel_nb_words'] = data.nb_words / data.nb_words.max()
data['nb_chars'] = data.Tokens.str.len()
data['rel_nb_chars'] = data.nb_chars / data.nb_chars.max()


data.head()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


train


,PhraseId,SentenceId,Text,Score,Tokens,nb_words,rel_nb_words,nb_chars,rel_nb_chars
0,32387,1518,really good,4,really good,2,0.038462,11,0.039427
1,61509,3110,vistas,2,vista,1,0.019231,5,0.017921
2,7990,327,and not worth,2,and not worth,3,0.057692,13,0.046595
3,65074,3292,unstinting look,2,unstinting look,2,0.038462,15,0.053763
4,102946,5424,"A thoroughly enjoyable , heartfelt",4,"a thoroughly enjoyable , heartfelt",5,0.096154,34,0.121864


In [27]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

oneword_sentiment = {}
df_len1 = data[data.nb_words == 1]
for label in df_len1.Score.value_counts().to_dict().keys():
    for word in df_len1.Tokens[df_len1.Score == label]:
        oneword_sentiment[word] = label


def vader_sentinet(word):
    vader_scores = sid.polarity_scores(word)
    v_neg = vader_scores['neg']
    v_neu = vader_scores['neu']
    v_pos = vader_scores['pos']
    
    senti_net = list(swn.senti_synsets(word))
    if len(senti_net) > 0:
        s_neg = senti_net[0].neg_score()
        s_pos = senti_net[0].pos_score()
        s_obj = senti_net[0].obj_score()
    else:
        s_neg, s_pos, s_obj = 0, 0, 0
    return v_neg, v_neu, v_pos, s_neg, s_pos, s_obj

def mean_senti_vader_score(text):
    if type(text) == str:
        text = text.split()
    score_lists = [[] for _ in range(6)]
    for w in text:
        scores = vader_sentinet(w)
        for i, s in enumerate(scores):
            score_lists[i].append(s)
    mean_scores = [sum(l) / max(1, len(l)) for l in score_lists]
    return mean_scores
        
def nb_senti_words(text):
    if type(text) == str:
        text = text.split()
    d = {-1 : 0, 0 : 0, 1 : 0, 2 : 0, 3 : 0, 4 : 0}
    for w in text:
        if w in oneword_sentiment:
            d[oneword_sentiment[w]] += 1
    for k in d:
        d[k] /= max(1, len(text))
        
    return d[0], d[1], d[2], d[3], d[4], d[-1]

print('train')
data = data.merge(data.Tokens.apply(lambda t: pd.Series(nb_senti_words(t))), 
                left_index=True, right_index=True)
data = data.rename(columns={
                        0 : "nb_senti_0", 
                        1 : "nb_senti_1", 
                        2 : "nb_senti_2", 
                        3 : "nb_senti_3",
                        4 : "nb_senti_4",
                        5 : "nb_senti_-1"
                       })

print('\tvader/sentinet')
data = data.merge(data.Tokens.apply(lambda t: pd.Series(mean_senti_vader_score(t))), 
                left_index=True, right_index=True)
data = data.rename(columns={
                        0 : "v_neg", 
                        1 : "v_neu", 
                        2 : "v_pos", 
                        3 : "s_neg",
                        4 : "s_pos",
                        5 : "s_obj"
                       })

data.head()

train
	vader/sentinet


,PhraseId,SentenceId,Text,Score,Tokens,nb_words,rel_nb_words,nb_chars,rel_nb_chars,nb_senti_0,nb_senti_1,nb_senti_2,nb_senti_3,nb_senti_4,nb_senti_-1,nb_senti_0,nb_senti_1,nb_senti_2,nb_senti_3,nb_senti_4,nb_senti_-1,nb_senti_0,nb_senti_1,nb_senti_2,nb_senti_3,nb_senti_4,nb_senti_-1,v_neg,v_neu,v_pos,s_neg,s_pos,s_obj
0,32387,1518,really good,4,really good,2,0.038462,11,0.039427,0.0,0.000000,0.500000,0.5,0.0,0.0,0.0,0.000000,0.500000,0.5,0.0,0.0,0.0,0.000000,0.500000,0.5,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.5625,0.437500
1,61509,3110,vistas,2,vista,1,0.019231,5,0.017921,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0000,1.000000
2,7990,327,and not worth,2,and not worth,3,0.057692,13,0.046595,0.0,0.333333,0.666667,0.0,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.0,0.666667,0.333333,0.208333,0.0000,0.458333
3,65074,3292,unstinting look,2,unstinting look,2,0.038462,15,0.053763,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0000,1.000000
4,102946,5424,"A thoroughly enjoyable , heartfelt",4,"a thoroughly enjoyable , heartfelt",5,0.096154,34,0.121864,0.0,0.000000,0.600000,0.4,0.0,0.0,0.0,0.000000,0.600000,0.4,0.0,0.0,0.0,0.000000,0.600000,0.4,0.0,0.0,0.0,0.200000,0.400000,0.050000,0.2250,0.525000


In [29]:
from nltk.corpus import subjectivity, movie_reviews
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC

print('subjectivity')
subj_docs = []
for l in tqdm(['subj', 'obj']):
    for s in subjectivity.sents(categories=l):
        subj_docs.append((l, tockenize_(s)))

print('polarity')
review_docs = []
for l in tqdm(['pos', 'neg']):
    for s in movie_reviews.sents(categories=l):
        review_docs.append((l, tockenize_(s)))

subj_df = pd.DataFrame(subj_docs, columns = ['label', 'text'])
review_df = pd.DataFrame(review_docs, columns = ['label', 'text'])

subj_cv = CountVectorizer(binary = True, min_df = 5, max_df = .5, dtype = np.int8).fit(subj_df.text)
review_cv = CountVectorizer(binary = True, min_df = 5, max_df = .5, dtype = np.int8).fit(review_df.text)

X_subj = subj_cv.transform(subj_df.text)
X_review = review_cv.transform(review_df.text)

print('subj_logit')
subj_logit = LogisticRegressionCV(n_jobs = 3, max_iter = 400, 
                                  cv=6, random_state=0, verbose = True).fit(X_subj, subj_df.label)
print('review_logit')
review_logit = LogisticRegressionCV(n_jobs = 3, max_iter = 400, 
                                  cv=6, random_state=0, verbose = True).fit(X_review, review_df.label)

print('subj', subj_logit.score(X_subj, subj_df.label))
print('polarity', review_logit.score(X_review, review_df.label))

data['subj_0'], data['subj_1'] = zip(*subj_logit.predict_proba(subj_cv.transform(data.Tokens)))
data['review_0'], data['review_1'] = zip(*review_logit.predict_proba(review_cv.transform(data.Tokens)))

del subj_df, subj_logit, subj_cv
del review_df, review_logit, review_cv

data.head()


subjectivity



polarity



subj_logit


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   6 out of   6 | elapsed:   10.8s finished


review_logit


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   6 out of   6 | elapsed:  2.1min finished


subj 0.9552
polarity 0.7647346642062294


,PhraseId,SentenceId,Text,Score,Tokens,nb_words,rel_nb_words,nb_chars,rel_nb_chars,nb_senti_0,nb_senti_1,nb_senti_2,nb_senti_3,nb_senti_4,nb_senti_-1,nb_senti_0,nb_senti_1,nb_senti_2,nb_senti_3,nb_senti_4,nb_senti_-1,nb_senti_0,nb_senti_1,nb_senti_2,nb_senti_3,nb_senti_4,nb_senti_-1,v_neg,v_neu,v_pos,s_neg,s_pos,s_obj,subj_0,subj_1,review_0,review_1
0,32387,1518,really good,4,really good,2,0.038462,11,0.039427,0.0,0.000000,0.500000,0.5,0.0,0.0,0.0,0.000000,0.500000,0.5,0.0,0.0,0.0,0.000000,0.500000,0.5,0.0,0.0,0.0,0.500000,0.500000,0.000000,0.5625,0.437500,0.208254,0.791746,0.513218,0.486782
1,61509,3110,vistas,2,vista,1,0.019231,5,0.017921,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0000,1.000000,0.490048,0.509952,0.555898,0.444102
2,7990,327,and not worth,2,and not worth,3,0.057692,13,0.046595,0.0,0.333333,0.666667,0.0,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.0,0.666667,0.333333,0.208333,0.0000,0.458333,0.262924,0.737076,0.473520,0.526480
3,65074,3292,unstinting look,2,unstinting look,2,0.038462,15,0.053763,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0000,1.000000,0.382419,0.617581,0.645582,0.354418
4,102946,5424,"A thoroughly enjoyable , heartfelt",4,"a thoroughly enjoyable , heartfelt",5,0.096154,34,0.121864,0.0,0.000000,0.600000,0.4,0.0,0.0,0.0,0.000000,0.600000,0.4,0.0,0.0,0.0,0.000000,0.600000,0.4,0.0,0.0,0.0,0.200000,0.400000,0.050000,0.2250,0.525000,0.219345,0.780655,0.451776,0.548224


In [31]:
def vader_score(text):
    scores = sid.polarity_scores(text)
    return scores['neg'], scores['neu'], scores['pos'], scores['compound']

print('train')
data = data.merge(data.Tokens.apply(lambda t: pd.Series(vader_score(t))), 
         left_index=True, right_index=True)
data = data.rename(columns={0 : "vader_neg", 1 : "vader_neu", 2 : "vader_pos", 3 : "vader_compound"})

train


In [33]:
from scipy.sparse import vstack, hstack, csr_matrix
from scipy.sparse.linalg import svds

print('data')
cv = CountVectorizer(binary = True, max_df = .5, min_df = 5, 
                        dtype = np.int8).fit(data.Tokens)
#cv = TfidfVectorizer(max_df = .5, min_df = 5, dtype = np.float32).fit(df.Tokens)

X_text = cv.transform(data.Tokens) #.toarray()

print('starting shape:', X_text.shape)

data
starting shape: (124848, 12710)


In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder


tockenizer_app = Tokenizer()
tockenizer_app.fit_on_texts(data.Tokens)


X_seq_list = tockenizer_app.texts_to_sequences(data.Tokens)
# seq_len = max(max([len(s) for s in X_seq_list]))

# X_seq = pad_sequences(X_seq_list, maxlen=len(X_seq_list))


NameError: ignored

In [1]:
s = 1
max(X_seq_list(1))

NameError: ignored